# Bacterial Genome-Wide Association Study (GWAS) using kmers

## 1. Preliminaries

Run the initialization script. Note the [\*] on the left means the script is running. When finished, the \* will be replaced with a number. Wait until it is finished. The script will also say when it is done.

If there is an error, remove the # from the second line and try again.

In [25]:
%%bash
# Bash code
#chmod -R 755 ~/Wednesday/ && rm -rf ~/Wednesday
bash ~/init.sh

Done !


Set your workshop identifier.

In [6]:
if 'PHENO' in globals():
    print('Your participant id is already: ' + PHENO)
else:
    import random
    PHENO = 'workshop' + str(random.randint(1, 35)).zfill(2)
    print('Your participant id is: ' + PHENO)

Your participant id is: workshop12


Define the location of input and output files as python variables.

In [7]:
# Location of the kmer_pipeline scripts
SCRIPTS='/usr/local/bin'
# Location of the S. aureus reference genomes
REFDIR='/usr/share/kmer_pipeline/example'
# File containing locations of software apps
APPS=REFDIR+'/pipeline_software_location.txt'
# Parent folder for the amr input/output data
AMRDIR='/home/jovyan/Wednesday/amr'
# Folder containing pre-computed genome data
INDIR=AMRDIR+'/take1/take1_cip_nucleotide31'
# Folder containing downloaded phenotypes
PHENO_DIR=AMRDIR+'/phenotypes'
# Folder for output of data analysis
OUTDIR=AMRDIR+'/out'
# Folder for storing error logs
LOGDIR=OUTDIR+'/log'
# Prefix for output files
OUTPUT_PREFIX='amr'
# Number of cores available per user
NCORES='2'

## 2. Run Genome-wide Efficient Mixed Model Associations (GEMMA)
Xiang Zhou and Matthew Stephens (2012). Genome-wide efficient mixed-model analysis for association studies. *Nature Genetics* 44, 821–824.

The following executes a bash script that first runs GEMMA in parallel then monitors the output to see when it is finished. It takes about 15 minutes to complete on two cores.

In [30]:
%%script env SCRIPTS="$SCRIPTS" REFDIR="$REFDIR" APPS="$APPS" AMRDIR="$AMRDIR" INDIR="$INDIR" PHENO_DIR="$PHENO_DIR" OUTDIR="$OUTDIR" LOGDIR="$LOGDIR" OUTPUT_PREFIX="$OUTPUT_PREFIX" NCORES="$NCORES" PHENO="$PHENO" bash
# Bash script
#!/usr/bin/env
# Create the output directories
mkdir -p $OUTDIR
mkdir -p $LOGDIR
cd $OUTDIR
# Parallelize over multiple cores
for SGE_TASK_ID in $(seq 1 $NCORES); do
    # Remove previous log files
    rm -f $LOGDIR/rungemma.$SGE_TASK_ID.log
    # Execute the Rscript wrapper for GEMMA
    SGE_TASK_ID=$SGE_TASK_ID \
        $SCRIPTS/rungemma.Rscript $NCORES $INDIR \
        $PHENO_DIR/$PHENO.txt $OUTPUT_PREFIX.$PHENO \
        $OUTDIR nucleotide 31 $APPS &> \
        $LOGDIR/rungemma.$SGE_TASK_ID.log &
    # Record process ID
    PIDS[${SGE_TASK_ID}]=$!
done
echo 'GEMMA scripts launched ...'

GEMMA scripts launched ...


In [23]:
%%script env LOGDIR="$LOGDIR" bash
echo ' '
echo '1a. Open a new Terminal tab (select from the menu File->New Terminal)'
echo '1b. Copy and paste the following line to track the progress of the first job:'
echo ' '
echo '        tail --follow -n +1 '$LOGDIR'/rungemma.1.log'
echo ' '
echo ' '
echo '2a. Open a second Terminal tab in the same way'
echo '2b. Copy and paste the following code to track the progress of the second job:'
echo ' '
echo '        tail --follow -n +1 '$LOGDIR'/rungemma.2.log'
echo ' '
echo ' '

 
1a. Open a new Terminal tab (select from the menu File->New Terminal)
1b. Copy and paste the following line to track the progress of the first job:
 
        tail --follow -n +1 /home/jovyan/Wednesday/amr/out/log/rungemma.1.log
 
 
2a. Open a second Terminal tab in the same way
2b. Copy and paste the following code to track the progress of the second job:
 
        tail --follow -n +1 /home/jovyan/Wednesday/amr/out/log/rungemma.2.log
 
 


Monitor the Terminal sessions to see when the processes have both finished.

When **both jobs have finished**, move on to the next section.
This takes about 15 minutes.

## 3. Create Figures to Interpret Results

**Warning**: executing the code below prematurely will terminate Step 2, requiring it to be re-run. If this happens, you should see 'No such file or directory' errors in the Step 2 log files.

In [31]:
%%script env SCRIPTS="$SCRIPTS" REFDIR="$REFDIR" APPS="$APPS" AMRDIR="$AMRDIR" INDIR="$INDIR" PHENO_DIR="$PHENO_DIR" OUTDIR="$OUTDIR" LOGDIR="$LOGDIR" OUTPUT_PREFIX="$OUTPUT_PREFIX" NCORES="$NCORES" PHENO="$PHENO" bash
# Bash script: press Shift+Enter
#!/usr/bin/env
# Execute the Rscript that produces the figures
cd $OUTDIR
SGE_TASK_ID=1; SGE_TASK_ID=$SGE_TASK_ID \
    $SCRIPTS/plotManhattanbowtie.Rscript \
    $OUTPUT_PREFIX.$PHENO $OUTDIR $INDIR \
    $REFDIR/R00000022.gbk $REFDIR/R00000022.fa \
    $PHENO_DIR/$PHENO.txt nucleotide 31 0.01 10 $APPS \
    70 &> $LOGDIR/plotManhattanbowtie.$SGE_TASK_ID.log &

In [32]:
%%script env LOGDIR="$LOGDIR" bash
echo ''
echo '3a. Open a new Terminal tab (select from the menu File->New Terminal)'
echo '3b. Copy and paste the following line to track the progress of the first job:'
echo ' '
echo '        tail --follow -n +1 '$LOGDIR'/plotManhattanbowtie.1.log'
echo ' '
echo ' '


3a. Open a new Terminal tab (select from the menu File->New Terminal)
3b. Copy and paste the following line to track the progress of the first job:
 
        tail --follow -n +1 /home/jovyan/Wednesday/amr/out/log/plotManhattanbowtie.1.log
 
 


4a. Once finished, select the Folder icon on the Left Siderbar, if it is not already open.

4b. Navigate to Wednesday/amr/out/nucleotidekmer31_bowtie2mapping_figures
